In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
# Crear una SparkSession
spark = SparkSession.builder \
    .appName("ejemplo") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

In [2]:
# Creation du Dataframe
data = [
    ('Jean', 1234, '2023-01-01', 'Homme'),
    ('Marie', 9876, '2023-02-15', 'Femme'),
    ('Paul', 1234, '2023-03-10', 'Homme'),
    ('Lucie', 1231, '2023-04-05', 'Femme'),
    ('Pierre', 4567, '2023-05-20', 'Homme')
]

columns = ['nom', 'id', 'date', 'sexe']

df = spark.createDataFrame(data, columns)

df

nom,id,date,sexe
Jean,1234,2023-01-01,Homme
Marie,9876,2023-02-15,Femme
Paul,1234,2023-03-10,Homme
Lucie,1231,2023-04-05,Femme
Pierre,4567,2023-05-20,Homme


In [3]:
# 1. `lit`: Ajouter une colonne avec une valeur constante
df = df.withColumn('valeur_constante', F.lit(1))

df

nom,id,date,sexe,valeur_constante
Jean,1234,2023-01-01,Homme,1
Marie,9876,2023-02-15,Femme,1
Paul,1234,2023-03-10,Homme,1
Lucie,1231,2023-04-05,Femme,1
Pierre,4567,2023-05-20,Homme,1


In [4]:
# 2. `col` et `when`: Créer une colonne avec une condition
df = df.withColumn('longueur_nom', F.when(F.length('nom') > 4, 'Long').otherwise('Court'))
df


nom,id,date,sexe,valeur_constante,longueur_nom
Jean,1234,2023-01-01,Homme,1,Court
Marie,9876,2023-02-15,Femme,1,Long
Paul,1234,2023-03-10,Homme,1,Court
Lucie,1231,2023-04-05,Femme,1,Long
Pierre,4567,2023-05-20,Homme,1,Long


In [5]:
# 3. `concat`: Combiner des colonnes en une seule chaîne
df = df.withColumn('nom_id', F.concat(F.col('nom'), F.lit('_'), F.col('id')))
df

nom,id,date,sexe,valeur_constante,longueur_nom,nom_id
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567


In [6]:
# 4. `month`, `year`, `quarter`: Extraire le mois, l'année et le trimestre
df = df.withColumn('mois', F.month('date')) \
       .withColumn('année', F.year('date')) \
       .withColumn('trimestre', F.quarter('date'))
df

nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2


In [7]:
# 5. `abs` et `floor`: Appliquer des fonctions mathématiques
df = df.withColumn('id_absolu', F.abs(F.col('id'))) \
       .withColumn('id_floor', F.floor(F.col('id') / 1000))
df

nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4


In [8]:
# 6. `concat_ws`: Concaténer avec un séparateur
df = df.withColumn('concat_ws', F.concat_ws(':', F.col('nom'), F.col('id')))
df

nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567


In [9]:
# 7. `array_distinct`, `collect_list`, `collect_set`: Utiliser des fonctions d'agrégation sur des arrays
df_agg = df.groupBy('sexe').agg(F.collect_list('nom').alias('noms'))
df_agg


sexe,noms
Homme,"[Jean, Paul, Pierre]"
Femme,"[Marie, Lucie]"


In [10]:
# 8. `split` et `trim`: Nettoyer et diviser des chaînes
df = df.withColumn('nom_nettoyé', F.trim(F.col('nom'))) \
       .withColumn('nom_divisé', F.split(F.col('nom'), ''))
df


nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]"
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]"
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]"
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]"
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]"


In [11]:
# 9. `explode`: Diviser une colonne de tableau en lignes distinctes
df_explode = df.withColumn('caractère_nom', F.explode(F.split(F.col('nom'), '')))
df_explode


nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,caractère_nom
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",J
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",e
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",a
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",n
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",M
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",a
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",r
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",i
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",e
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",P


In [12]:
# 10. `lower`: Convertir des chaînes en minuscules
df = df.withColumn('nom_minuscule', F.lower(F.col('nom')))
df


nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre


In [13]:
# 11. `select`: Sélectionner des colonnes spécifiques
df_select = df.select('nom', 'id', 'date')
df_select


nom,id,date
Jean,1234,2023-01-01
Marie,9876,2023-02-15
Paul,1234,2023-03-10
Lucie,1231,2023-04-05
Pierre,4567,2023-05-20


In [14]:
# 12. `withColumn`: Créer une nouvelle colonne
df = df.withColumn('nouvelle_colonne', F.lit('nouvelle_valeur'))
df

nom,id,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,nouvelle_colonne
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,nouvelle_valeur
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,nouvelle_valeur
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul,nouvelle_valeur
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie,nouvelle_valeur
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre,nouvelle_valeur


In [15]:
# 13. `withColumnRenamed`: Renommer une colonne
df = df.withColumnRenamed('id', 'identifiant')
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,nouvelle_colonne
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,nouvelle_valeur
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,nouvelle_valeur
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul,nouvelle_valeur
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie,nouvelle_valeur
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre,nouvelle_valeur


In [16]:
# 14. `drop`: Supprimer une colonne
df = df.drop('nouvelle_colonne')
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre


In [17]:
# 15. `join`: Joindre deux DataFrames (ici nous réutilisons le DataFrame lui-même pour l'exemple)
df_join = df.join(df, on='nom', how='inner')
df_join


nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre


In [18]:
# 16. `groupBy` et `agg`: Grouper par une colonne et appliquer une agrégation
df_group = df.groupBy('nom').agg(F.count('identifiant').alias('compte'))
df_group


nom,compte
Jean,1
Marie,1
Paul,1
Lucie,1
Pierre,1


In [19]:
# 17. `drop_duplicates`: Supprimer les doublons
df_no_double = df.dropDuplicates()
df_no_double


nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre


In [20]:
# 18. `where`/`filter`: Filtrer les lignes selon une condition
df_filter = df.filter(F.col('id') > 1000)
df_filter


nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre


In [21]:
# 19. `when` et `otherwise`: Créer une colonne conditionnelle
df = df.withColumn('condition', F.when(F.col('identifiant') > 2000, 'Haut').otherwise('Bas'))
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul,Bas
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie,Bas
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre,Haut


In [22]:
# 20. `astype`: Changer le type de colonne (équivalent à `cast`)
df = df.withColumn('id_str', F.col('identifiant').cast('string'))
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut,9876
Paul,1234,2023-03-10,Homme,1,Court,Paul_1234,3,2023,1,1234,1,Paul:1234,Paul,"[P, a, u, l]",paul,Bas,1234
Lucie,1231,2023-04-05,Femme,1,Long,Lucie_1231,4,2023,2,1231,1,Lucie:1231,Lucie,"[L, u, c, i, e]",lucie,Bas,1231
Pierre,4567,2023-05-20,Homme,1,Long,Pierre_4567,5,2023,2,4567,4,Pierre:4567,Pierre,"[P, i, e, r, r, e]",pierre,Haut,4567


In [23]:
# 21. `isin`: Vérifier si une valeur est dans une liste
df = df.filter(F.col('nom').isin('Jean', 'Marie'))
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut,9876


In [24]:
# 22. `lower`: Mettre des chaînes en minuscules (déjà utilisé mais réutilisé ici)
df = df.withColumn('nom_minuscule', F.lower(F.col('nom')))
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut,9876


In [25]:
# 23. `alias`: Utiliser un alias pour une colonne
df_alias = df.select(F.col('nom').alias('nom_alias'))
df_alias


nom_alias
Jean
Marie


In [26]:
# 24. `isNull` et `isNotNull`: Vérifier si une colonne est nulle ou non
df = df.filter(F.col('nom').isNotNull())
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut,9876


In [27]:
# 25. `like` et `rlike`: Filtrer les chaînes avec un modèle
df_like = df.filter(F.col('nom').like('%ean%'))
df_like


nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234


In [28]:
# 26. `regexp_replace`: Remplacer des motifs dans une chaîne avec des expressions régulières
df = df.withColumn('nom_modifié', F.regexp_replace(F.col('nom'), 'Jean', 'Jean-Paul'))
df

nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str,nom_modifié
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234,Jean-Paul
Marie,9876,2023-02-15,Femme,1,Long,Marie_9876,2,2023,1,9876,9,Marie:9876,Marie,"[M, a, r, i, e]",marie,Haut,9876,Marie


In [29]:
# 27. `str.contains`: Vérifier si une chaîne contient un motif (équivalent à `like`)
df_contains = df.filter(F.col('nom').contains('Jean'))
df_contains


nom,identifiant,date,sexe,valeur_constante,longueur_nom,nom_id,mois,année,trimestre,id_absolu,id_floor,concat_ws,nom_nettoyé,nom_divisé,nom_minuscule,condition,id_str,nom_modifié
Jean,1234,2023-01-01,Homme,1,Court,Jean_1234,1,2023,1,1234,1,Jean:1234,Jean,"[J, e, a, n]",jean,Bas,1234,Jean-Paul


In [30]:
# differences performances entre collect() - show()
import time

# Comparaison entre collect() et show()
start = time.time()
df.collect()
end = time.time()
print(f"Temps pour collect(): {end - start} seconds")

start = time.time()
df.show()
end = time.time()
print(f"Temps pour show(): {end - start} seconds")


Temps pour collect(): 5.636657238006592 seconds
+-----+-----------+----------+-----+----------------+------------+----------+----+-----+---------+---------+--------+----------+-----------+---------------+-------------+---------+------+-----------+
|  nom|identifiant|      date| sexe|valeur_constante|longueur_nom|    nom_id|mois|année|trimestre|id_absolu|id_floor| concat_ws|nom_nettoyé|     nom_divisé|nom_minuscule|condition|id_str|nom_modifié|
+-----+-----------+----------+-----+----------------+------------+----------+----+-----+---------+---------+--------+----------+-----------+---------------+-------------+---------+------+-----------+
| Jean|       1234|2023-01-01|Homme|               1|       Court| Jean_1234|   1| 2023|        1|     1234|       1| Jean:1234|       Jean|   [J, e, a, n]|         jean|      Bas|  1234|  Jean-Paul|
|Marie|       9876|2023-02-15|Femme|               1|        Long|Marie_9876|   2| 2023|        1|     9876|       9|Marie:9876|      Marie|[M, a, r, i,

In [31]:
# differences performances entre RDD - count()
start = time.time()
df_rdd_count = df.rdd.count()
end = time.time()
print(f"Temps pour RDD count(): {end - start} seconds")

start = time.time()
df_df_count = df.count()
end = time.time()
print(f"Temps pour DataFrame count(): {end - start} seconds")


Temps pour RDD count(): 11.384246826171875 seconds
Temps pour DataFrame count(): 5.4931230545043945 seconds
